<a href="https://colab.research.google.com/github/Gaurav0502/electricity-forecasting/blob/eeshan/LSTM_TSAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Gaurav0502/electricity-forecasting

Cloning into 'electricity-forecasting'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 129 (delta 63), reused 91 (delta 39), pack-reused 0 (from 0)
Receiving objects: 100% (129/129), 2.10 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [2]:
%cd electricity-forecasting

/content/electricity-forecasting


In [3]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/balaganesh
  remotes/origin/eeshan
  remotes/origin/gaurav
  remotes/origin/jason
  remotes/origin/main


In [4]:
!git checkout eeshan

Branch 'eeshan' set up to track remote branch 'eeshan' from 'origin'.
Switched to a new branch 'eeshan'


In [5]:
!git pull origin main

From https://github.com/Gaurav0502/electricity-forecasting
 * branch            main       -> FETCH_HEAD
Updating c3e5a97..1194267
Fast-forward
 .github/CONTRIBUTING.md |  45 ++++++--
 clustering.ipynb        | 350 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 clusters.json           | 339 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 eda.ipynb               | 405 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++-----------
 model.py                | 193 ++++++++++++++++++++++++++++++++
 modelling.ipynb         | 279 +++++++++++++++++++++++++++++++++++++++++++++++
 preprocess.py           |  62 ++++++++++-
 requirements.txt        |   8 +-
 8 files changed, 1607 insertions(+), 74 deletions(-)
 create mode 100644 clustering.ipynb
 create mode 100644 clusters.json
 create mode 100644 model.py
 create mode 100644 modelling.ipynb


In [6]:
!cat preprocess.py

import pandas as pd
import numpy as np
import json

DATA_DIR = {
  "electricity": "data/LD2011_2014.txt",
  "temperature": "data/lisbon_temp_2011-2014.csv",
  "precipitation": "data/lisbon_precip_2011-2014.csv"
}

YEARS = [2011, 2012, 2013, 2014]

# replaces comma with dot
def str2float(x: str):
    return float(str(x).replace(",", "."))

def preprocess_electricity():

    df = pd.read_csv(DATA_DIR["electricity"], sep=";")

    # giving a name to the time column
    df = df.rename({"Unnamed: 0": "timestamp"}, axis = 1)

    # converting object type cols to float64
    cols = [i for i in df.columns if i != "timestamp"]
    for col in cols:
        df[col] = df[col].apply(str2float)
    
    return df

def preprocess_weather(dataset_name: str):

    df = pd.read_csv(DATA_DIR[dataset_name])

    if dataset_name == "precipitation":
       df = df.rename(columns = {"Year": "year",
                                 "Month": "month",
                                 "Day": "day",
             

In [7]:
!mkdir -p data

In [8]:
!cp /content/LD2011_2014.txt data/
!cp /content/lisbon_precip_2011-2014.csv data/
!cp /content/lisbon_temp_2011-2014.csv data/

In [9]:
import preprocess
electricity = preprocess.preprocess_modelling(as_numpy=False, clustered=True, stationary=True)

/content/electricity-forecasting/preprocess.py:19: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,125,126,127,128,129,130,131,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,157,160,164,165,167,170,173,177,178,179,181,184,185,186,223,224,255,280,289,305,308,322,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,363,364,365,366,367,368,369,370) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_DIR["electricity"], sep=";")
/content/electricity-forecasting/preprocess.py:87: Per

In [10]:
temp = preprocess.preprocess("temperature")
precip = preprocess.preprocess("precipitation")

In [11]:
electricity.head()

,cluster_0,cluster_1
date,,
2011-01-01,NaN,NaN
2011-01-02,NaN,NaN
2011-01-03,-27879.431083,-4.152764e+06
2011-01-04,806.030855,1.102588e+05
2011-01-05,141.590190,-1.650482e+05


In [12]:
temp.head()

,tmin,tmax,date
0,12.5,17.6,2011-01-01
1,10.0,17.2,2011-01-02
2,9.7,12.0,2011-01-03
3,9.6,13.1,2011-01-04
4,11.5,15.5,2011-01-05


In [13]:
precip.head()

,precip,date
0,0.6,2011-01-01
1,0.4,2011-01-02
2,0.0,2011-01-03
3,0.0,2011-01-04
4,3.6,2011-01-05


In [14]:
temp.info()
precip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   tmin    1461 non-null   float64       
 1   tmax    1461 non-null   float64       
 2   date    1461 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 34.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   precip  1461 non-null   float64       
 1   date    1461 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 23.0 KB


In [15]:
temp.isna().sum()

,0
tmin,0
tmax,0
date,0


In [16]:
precip.isna().sum()

,0
precip,0
date,0


In [17]:
temp.describe()

,tmin,tmax,date
count,1461.000000,1461.000000,1461
mean,13.794524,21.355373,2012-12-31 00:00:00
min,2.700000,8.000000,2011-01-01 00:00:00
25%,10.200000,16.400000,2012-01-01 00:00:00
50%,14.000000,20.900000,2012-12-31 00:00:00
75%,17.400000,26.000000,2013-12-31 00:00:00
max,24.400000,37.800000,2014-12-31 00:00:00
std,4.368122,5.699943,NaN


In [18]:
precip.describe()

,precip,date
count,1461.000000,1461
mean,2.719097,2012-12-31 00:00:00
min,0.000000,2011-01-01 00:00:00
25%,0.000000,2012-01-01 00:00:00
50%,0.000000,2012-12-31 00:00:00
75%,0.400000,2013-12-31 00:00:00
max,102.000000,2014-12-31 00:00:00
std,8.495627,NaN


In [19]:
print("Temp date range:", temp["date"].min(), "to", temp["date"].max())
print("Precip date range:", precip["date"].min(), "to", precip["date"].max())

Temp date range: 2011-01-01 00:00:00 to 2014-12-31 00:00:00
Precip date range: 2011-01-01 00:00:00 to 2014-12-31 00:00:00


In [20]:
electricity.index

Index([2011-01-01, 2011-01-02, 2011-01-03, 2011-01-04, 2011-01-05, 2011-01-06,
       2011-01-07, 2011-01-08, 2011-01-09, 2011-01-10,
       ...
       2014-12-23, 2014-12-24, 2014-12-25, 2014-12-26, 2014-12-27, 2014-12-28,
       2014-12-29, 2014-12-30, 2014-12-31, 2015-01-01],
      dtype='object', name='date', length=1462)

In [21]:
import pandas as pd

In [22]:
electricity.index = pd.to_datetime(electricity.index)

In [23]:
electricity.index

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03', '2011-01-04',
               '2011-01-05', '2011-01-06', '2011-01-07', '2011-01-08',
               '2011-01-09', '2011-01-10',
               ...
               '2014-12-23', '2014-12-24', '2014-12-25', '2014-12-26',
               '2014-12-27', '2014-12-28', '2014-12-29', '2014-12-30',
               '2014-12-31', '2015-01-01'],
              dtype='datetime64[ns]', name='date', length=1462, freq=None)

In [24]:
electricity.isna().sum()

,0
cluster_0,2
cluster_1,2


In [25]:
electricity.head()

,cluster_0,cluster_1
date,,
2011-01-01,NaN,NaN
2011-01-02,NaN,NaN
2011-01-03,-27879.431083,-4.152764e+06
2011-01-04,806.030855,1.102588e+05
2011-01-05,141.590190,-1.650482e+05


In [26]:
print("Electricity date range:", electricity.index.min(), "to", electricity.index.max())

Electricity date range: 2011-01-01 00:00:00 to 2015-01-01 00:00:00


In [27]:
electricity.describe()

,cluster_0,cluster_1
count,1.460000e+03,1.460000e+03
mean,-4.711479e+03,-8.659687e+03
std,5.822306e+05,7.681457e+05
min,-6.179482e+06,-7.643785e+06
25%,-1.479642e+05,-2.462727e+05
50%,-2.714979e+02,-7.955609e+02
75%,1.384833e+05,2.268773e+05
max,6.146092e+06,9.075160e+06


In [28]:
electricity.dropna(inplace=True)

In [29]:
start_date = electricity.index[0]
print("Starting index date:", start_date)
end_date = '2014-12-31'

Starting index date: 2011-01-03 00:00:00


In [30]:
electricity = electricity.loc[start_date:end_date]

In [31]:
temp = temp[(temp["date"] >= start_date) & (temp["date"] <= end_date)]
precip = precip[(precip["date"] >= start_date) & (precip["date"] <= end_date)]

In [32]:
# If electricity's index is the date, reset it:
electricity = electricity.reset_index().rename(columns={"index": "date"})

# Now merge on "date"
merged_df = electricity.merge(temp, on="date", how="inner").merge(precip, on="date", how="inner")

# Optionally set "date" back as the index
merged_df.set_index("date", inplace=True)

In [33]:
merged_df.head()

,cluster_0,cluster_1,tmin,tmax,precip
date,,,,,
2011-01-03,-27879.431083,-4.152764e+06,9.7,12.0,0.0
2011-01-04,806.030855,1.102588e+05,9.6,13.1,0.0
2011-01-05,141.590190,-1.650482e+05,11.5,15.5,3.6
2011-01-06,1851.419797,1.769558e+04,14.4,16.5,28.8
2011-01-07,-2784.062738,-1.470647e+05,14.3,17.6,17.4


In [34]:
merged_df.tail()

,cluster_0,cluster_1,tmin,tmax,precip
date,,,,,
2014-12-27,-2.470207e+06,-4.703764e+06,9.4,15.1,0.0
2014-12-28,-1.984328e+04,-2.313097e+05,10.8,14.1,0.0
2014-12-29,-3.748185e+04,2.921974e+05,7.0,16.2,0.0
2014-12-30,-1.642309e+05,3.398126e+04,4.0,11.4,0.0
2014-12-31,-5.042978e+05,-8.533031e+05,4.5,11.2,0.0


In [35]:
merged_df.isna().sum()

,0
cluster_0,0
cluster_1,0
tmin,0
tmax,0
precip,0


In [36]:
merged_df.describe()

,cluster_0,cluster_1,tmin,tmax,precip
count,1.459000e+03,1.459000e+03,1459.000000,1459.000000,1459.000000
mean,-4.792857e+02,-3.426565e+03,13.798012,21.360795,2.722138
std,5.595144e+05,7.419178e+05,4.369854,5.701962,8.501054
min,-4.263428e+06,-4.703764e+06,2.700000,8.000000,0.000000
25%,-1.470790e+05,-2.460339e+05,10.200000,16.400000,0.000000
50%,-2.471634e+02,-4.607150e+02,14.000000,20.900000,0.000000
75%,1.389339e+05,2.268895e+05,17.400000,26.000000,0.400000
max,6.146092e+06,9.075160e+06,24.400000,37.800000,102.000000


In [37]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(merged_df)

# Convert back to a DataFrame with the same columns and index
merged_df_scaled = pd.DataFrame(scaled_values,
                                columns=merged_df.columns,
                                index=merged_df.index)

In [38]:
# Example split percentages (70% train, 15% val, 15% test)
train_size = int(len(merged_df_scaled) * 0.70)
val_size = int(len(merged_df_scaled) * 0.15)
test_size = len(merged_df_scaled) - train_size - val_size

train_df = merged_df_scaled.iloc[:train_size]
val_df = merged_df_scaled.iloc[train_size:train_size+val_size]
test_df = merged_df_scaled.iloc[train_size+val_size:]

In [39]:
import numpy as np

def create_sequences(data, target_col_index, window_size=7):
    """
    data: 2D NumPy array of shape (time, features)
    target_col_index: index of the target column(s) in data
    window_size: number of past days to use as input

    Returns:
        X: shape (num_samples, window_size, num_features)
        y: shape (num_samples,) or (num_samples, target_features)
    """
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size, target_col_index])
    return np.array(X), np.array(y)

In [40]:
feature_cols = ['cluster_0', 'cluster_1', 'tmin', 'tmax', 'precip']
target_col = 'cluster_0'

# Get column indices
feature_indices = [merged_df_scaled.columns.get_loc(c) for c in feature_cols]
target_index = merged_df_scaled.columns.get_loc(target_col)

train_array = train_df[feature_cols].values
X_train, y_train = create_sequences(train_array, target_index, window_size=7)

val_array = val_df[feature_cols].values
X_val, y_val = create_sequences(val_array, target_index, window_size=7)

test_array = test_df[feature_cols].values
X_test, y_test = create_sequences(test_array, target_index, window_size=7)

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(7, len(feature_cols))))
model.add(Dropout(0.2))
model.add(Dense(1))  # if you're predicting only cluster_0

model.compile(optimizer='adam', loss='mse')
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 64)                  │          17,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,985 (70.25 KB)

 Trainable params: 17,985 (70.25 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.1211 - val_loss: 0.0100
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - val_loss: 0.0038
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0069 - val_loss: 0.0029
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0045 - val_loss: 0.0026
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0049 - val_loss: 0.0025
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0044 - val_loss: 0.0026
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0036 - val_loss: 0.0026
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0040 - val_loss: 0.0026
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0040 - val_loss: 0.0026
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0030 - val_loss: 0.0028
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0046 - val_loss: 0.0026
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0037 - val_l

In [43]:
y_pred = model.predict(X_test)

# If you scaled your data, remember to invert the predictions
# if you want them back in the original scale.
# For example, if 'cluster_0' was your target, you'd do something like:
# y_pred_inverted = scaler.inverse_transform(...)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [44]:
import numpy as np

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-9))) * 100

test_mape = mape(y_test, y_pred)
print(f"Test MAPE: {test_mape:.2f}%")

Test MAPE: 11.58%


In [45]:
# Define feature columns and set target to cluster_1
feature_cols = ['cluster_0', 'cluster_1', 'tmin', 'tmax', 'precip']
target_col = 'cluster_1'  # Changed target to cluster_1

# Get column indices
feature_indices = [merged_df_scaled.columns.get_loc(c) for c in feature_cols]
target_index = merged_df_scaled.columns.get_loc(target_col)

# Create sequences for training, validation, and testing
train_array = train_df[feature_cols].values
X_train, y_train = create_sequences(train_array, target_index, window_size=7)

val_array = val_df[feature_cols].values
X_val, y_val = create_sequences(val_array, target_index, window_size=7)

test_array = test_df[feature_cols].values
X_test, y_test = create_sequences(test_array, target_index, window_size=7)

# Build and compile the LSTM model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
# First LSTM layer with increased units and returning sequences for stacking
model.add(LSTM(128, activation='tanh', return_sequences=True, input_shape=(7, len(feature_cols))))
model.add(Dropout(0.3))
# Second LSTM layer, this one outputs a single vector
model.add(LSTM(64, activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
# An intermediate Dense layer to learn higher-level representations
model.add(Dense(32, activation='relu'))
# Final output layer
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()  # Predicting only cluster_1

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32)

# Predict on test set
y_pred = model.predict(X_test)

# Inverse scaling step goes here if needed (using the placeholder method as before)
# For example:
# placeholder = np.zeros((len(y_pred), merged_df_scaled.shape[1]))
# placeholder[:, target_index] = y_pred.ravel()
# placeholder_inverted = scaler.inverse_transform(placeholder)
# y_pred_inverted = placeholder_inverted[:, target_index]

# Define MAPE function and compute Test MAPE
import numpy as np

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-9))) * 100

test_mape = mape(y_test, y_pred)
print(f"Test MAPE: {test_mape:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 7, 128)              │          68,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 7, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 120,129 (469.25 KB)

 Trainable params: 120,129 (469.25 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 7, 128)              │          68,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 7, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 120,129 (469.25 KB)

 Trainable params: 120,129 (469.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 59ms/step - loss: 0.0295 - val_loss: 0.0047
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0028 - val_loss: 0.0045
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0032 - val_loss: 0.0044
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0022 - val_loss: 0.0047
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0034 - val_loss: 0.0045
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0031 - val_loss: 0.0045
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0027 - val_loss: 0.0046
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0026 - val_loss: 0.0044
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0022 - val_loss: 0.0045
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0034 - val_loss: 0.0045
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0027 - val_loss: 0.0046
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0

In [46]:
merged_df_scaled

,cluster_0,cluster_1,tmin,tmax,precip
date,,,,,
2011-01-03,0.406892,0.039989,0.322581,0.134228,0.000000
2011-01-04,0.409648,0.349376,0.317972,0.171141,0.000000
2011-01-05,0.409584,0.329395,0.405530,0.251678,0.035294
2011-01-06,0.409748,0.342658,0.539171,0.285235,0.282353
2011-01-07,0.409303,0.330701,0.534562,0.322148,0.170588
...,...,...,...,...,...
2014-12-27,0.172267,0.000000,0.308756,0.238255,0.000000
2014-12-28,0.407664,0.324587,0.373272,0.204698,0.000000
2014-12-29,0.405969,0.362580,0.198157,0.275168,0.000000


In [47]:
y_pred

array([[0.3474031 ],
       [0.3462564 ],
       [0.345122  ],
       [0.34603572],
       [0.34553066],
       [0.34489825],
       [0.3462527 ],
       [0.34745938],
       [0.34706324],
       [0.3473092 ],
       [0.34733123],
       [0.3469862 ],
       [0.34731928],
       [0.34940597],
       [0.35043386],
       [0.34838226],
       [0.34804106],
       [0.34609213],
       [0.34431854],
       [0.34408426],
       [0.34445575],
       [0.34732315],
       [0.3483083 ],
       [0.3479645 ],
       [0.34796235],
       [0.34713528],
       [0.3460459 ],
       [0.34751654],
       [0.34751427],
       [0.34598905],
       [0.34611323],
       [0.34588876],
       [0.34620902],
       [0.3456972 ],
       [0.3466179 ],
       [0.34876567],
       [0.34704155],
       [0.34766045],
       [0.34705886],
       [0.34544933],
       [0.34629518],
       [0.34797192],
       [0.35079646],
       [0.35004893],
       [0.34631115],
       [0.34817758],
       [0.34699944],
       [0.345

In [48]:
y_test

array([0.34599819, 0.34180752, 0.33642156, 0.35077222, 0.35849487,
       0.32083631, 0.32327792, 0.37581252, 0.33364869, 0.36761362,
       0.37180822, 0.31794983, 0.32218423, 0.28051097, 0.36970397,
       0.34247295, 0.33129641, 0.35224978, 0.3590424 , 0.32332578,
       0.3257328 , 0.36149859, 0.35006113, 0.33151595, 0.35106786,
       0.36322573, 0.31938403, 0.30854893, 0.37061242, 0.33567073,
       0.34929173, 0.36373447, 0.34558717, 0.31607557, 0.30541441,
       0.3619766 , 0.38773252, 0.33131988, 0.34098585, 0.34362646,
       0.3445553 , 0.26566824, 0.41334448, 0.32603081, 0.33015549,
       0.31121005, 0.35657703, 0.35942665, 0.31189656, 0.40265617,
       0.34157537, 0.27110467, 0.36635848, 0.38986234, 0.33485226,
       0.28423709, 0.36862773, 0.37188433, 0.31148126, 0.32088245,
       0.33932243, 0.3551143 , 0.32463216, 0.37882915, 0.36377276,
       0.30948923, 0.3488449 , 0.34318209, 0.3313532 , 0.30546077,
       0.38184265, 0.32790943, 0.31402557, 0.37856024, 0.36813

In [49]:
merged_df_scaled.describe()

,cluster_0,cluster_1,tmin,tmax,precip
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,0.409524,0.341125,0.511429,0.448349,0.026688
std,0.053750,0.053844,0.201376,0.191341,0.083344
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.395441,0.323518,0.345622,0.281879,0.000000
50%,0.409546,0.341340,0.520737,0.432886,0.000000
75%,0.422917,0.357840,0.677419,0.604027,0.003922
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
y_test.min()

np.float64(0.0)

In [51]:
def create_sequences_multi_output(data, target_cols_indices, window_size=7):
    """
    data: 2D NumPy array of shape (time, features)
    target_cols_indices: list of column indices for cluster_0 and cluster_1
    window_size: number of past days to use as input

    Returns:
        X: shape (num_samples, window_size, num_features)
        y: shape (num_samples, len(target_cols_indices))
    """
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])  # shape (window_size, num_features)
        # Gather the future values for each target column
        y_future = data[i+window_size, target_cols_indices]  # shape (2,)
        y.append(y_future)
    return np.array(X), np.array(y)

In [52]:
feature_cols = ['cluster_0', 'cluster_1', 'tmin', 'tmax', 'precip']
# For multi-output, we also want to predict both cluster_0 and cluster_1
target_cols = ['cluster_0', 'cluster_1']

feature_indices = [merged_df_scaled.columns.get_loc(c) for c in feature_cols]
target_indices = [merged_df_scaled.columns.get_loc(c) for c in target_cols]

train_array = train_df[feature_cols].values  # shape (train_time, 5)
X_train, y_train = create_sequences_multi_output(train_array, target_indices, window_size=7)

val_array = val_df[feature_cols].values
X_val, y_val = create_sequences_multi_output(val_array, target_indices, window_size=7)

test_array = test_df[feature_cols].values
X_test, y_test = create_sequences_multi_output(test_array, target_indices, window_size=7)

In [53]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, activation='tanh', return_sequences=False, input_shape=(7, len(feature_cols))))
model.add(Dropout(0.2))
model.add(Dense(2))  # 2 outputs -> [pred_cluster_0, pred_cluster_1]

model.compile(optimizer='adam', loss='mse')
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 64)                  │          17,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,050 (70.51 KB)

 Trainable params: 18,050 (70.51 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0872 - val_loss: 0.0088
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0083 - val_loss: 0.0048
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0055 - val_loss: 0.0039
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0050 - val_loss: 0.0037
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0043 - val_loss: 0.0037
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0042 - val_loss: 0.0036
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0041 - val_loss: 0.0037
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0035 - val_loss: 0.0036
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0040 - val_loss: 0.0037
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0041 - val_loss: 0.0036
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0048 - val_loss: 0.0036
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0047 - 

In [55]:
y_pred = model.predict(X_test)
print("y_pred shape:", y_pred.shape)  # should be (num_samples, 2)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
y_pred shape: (213, 2)


In [56]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-9))) * 100

# Suppose y_test and y_pred are (num_samples, 2)
test_mape_cluster0 = mape(y_test[:, 0], y_pred[:, 0])
test_mape_cluster1 = mape(y_test[:, 1], y_pred[:, 1])

In [57]:
print(test_mape_cluster0)

11.569468175079017


In [58]:
print(test_mape_cluster1)

154849237.29371816


In [59]:
!git status

On branch eeshan
Your branch is ahead of 'origin/eeshan' by 33 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [60]:
!git add LSTM_TSAF.ipynb

fatal: pathspec 'LSTM_TSAF.ipynb' did not match any files


In [61]:
!ls

clustering.ipynb  data	     modelling.ipynb  preprocess.py  README.md
clusters.json	  eda.ipynb  model.py	      __pycache__    requirements.txt
